# Mass spring damper system - GP + MPC

## Introduction

The system that is considered in this example is a linear mass-spring-damper system, whose dynamics are given by

<img src="../images/mass_spring_damper.png" alt="drawing" width="800"/>

\begin{align}
& \dot{x_1} = x_2 \\
& \dot{x_2} = \frac{1}{m} (u - k x_1 - d x_2) \\
\end{align}

Therein, the state $x_1$ is the displacement of the mass $m$, the state $x_2$ is the velocity of the mass, the input $u$ is the applied force, $k$ is the spring constant and $d$ is the damping factor.

We assume that the states can be measured perfectly, i.e., we will not define a separate output equation.

One sampling period takes $15~\text{ms}$.



## Problem setup

In [1]:
# Add HILO-MPC to path. NOT NECESSARY if it was installed via pip.
import sys
sys.path.append('../../../')

import numpy as np
import numpy.matlib as mat

from hilo_mpc import Model, GPR, NMPC, SimpleControlLoop
import hilo_mpc.core.learning.kernels as kernels

### Model setup

In [2]:
# === System set-up ================================================================================================
system = Model(plot_backend = 'bokeh', name = 'Linear SMD')

# Set symbolic states and inputs
system.set_dynamical_states('x', 2)
system.set_inputs('u')

# Add dynamics equations to model
system.set_dynamical_equations(['x_1', 'u - 2 * x_0 - 0.8 * x_1'])

# Sampling time
Ts = 0.015  # Ts = 15 ms

# Set-up
system.setup(dt=Ts)

In the following, we want to design a simple model predictive controller (MPC) for the system while assuming that the dynamics are unknown. Therefore, we will learn the dynamics from input-output data using a Gaussian process (GP) to create the prediction model.

### Gaussian Process Models

To learn the dynamics of the considered, system we require input-output data of the system on which the GP can be trained.

In the following, we first compute a suitable control sequence for generating these data. The control actions are lower- and upper-bounded and drawn from a uniform random distribution. Each control action is applied for several sampling periods.

In [3]:
num_rndInputs = 300  # Number of random inputs to be used for generating the training dat
hold_rndInputs = 40  # Number of time steps for which an input value is applied

# Bound region within which inputs can be located
u_lb = -25  # Lower bound
u_ub = 25  # Upper bound

# Initialize random number generator for reproducibility
np.random.seed(0)

# Draw uniformly distributed random inputs for generating training data
u_train = np.random.uniform(u_lb, u_ub, (num_rndInputs, 1))

# Expand and shape vector of control inputs
u_train = mat.repmat(u_train, 1, hold_rndInputs)
u_train = np.reshape(u_train, (num_rndInputs * hold_rndInputs, 1))

Given the just computed control input sequence, we now compute the system's response to it (these are the measurement data).

In [4]:
# Initial condition for training data generation
x_0 = [0, 0]

# Initialize system
system.set_initial_conditions(x_0)

# Create data frame for storing simulation results of the system for training data generation
solution_sys = system.solution

# Simulate system
for ti in range(u_train.shape[0]):
    # Simulate one time step
    system.simulate(u=u_train[ti])

x_train = solution_sys.get_by_id('x')
t_train = solution_sys.get_by_id('t')

Provided now the input-ouput for the system, we need to build the training data set $D = \{ X, y \} $, consisting of training inputs stored in $X$ and training outputs stored in $y$, for the Gaussian process.

We will learn the discretized state differences $\Delta x = \begin{bmatrix} \Delta x_1 & \Delta x_2 \end{bmatrix}^T$ over one sampling period. Approximating $\dot{x_i}~,~~i \in \{1, 2 \}$ by the difference quotients $\frac{\Delta x_i}{T_s}~,~~ i \in \{ 1, 2 \}$, we can derive the following setting:

\begin{align}
\Delta x_1 = x_1^+ - x_1 = T_s \cdot f_1 (x_1, x_2, u) = \tilde{f}_1 (x_1, x_2, u) ~, ~~ \tilde{f}_1 \sim \mathcal{GP}_1 (m_1, k_1) \\
\Delta x_2 = x_2^+ - x_2 = T_s \cdot f_2 (x_1, x_2, u) = \tilde{f}_2 (x_1, x_2, u) ~, ~~ \tilde{f}_2 \sim \mathcal{GP}_2 (m_2, k_2) 
\end{align}

We define both GPs via a zero mean function $m_1(w) = m_2(w) = 0$ and the isotropic squared exponential (SE) kernel

\begin{align}
k(w, w') = \sigma_f^2 \exp \left ( - \frac{\lVert w - w' \rVert_2^2}{2 \ell^2} \right )~,
\end{align}

since it is a universal kernel. The GPs use different kernel function hyperparameters $\sigma_f^2$ (signal variance) and $\ell$ (length scale).

The training inputs $w$, stored in the design matrix $X$, are given by $w = \begin{bmatrix} x_1 & x_2 & u \end{bmatrix}$ and are shared between both GPs. The training outputs of the first GP $\mathcal{GP}_1$ are the discretized differences $\Delta x_1 = x_1^+ - x_1$ of the first state. The training outputs of the second GP $\mathcal{GP}_2$ are the discretized differences $\Delta x_2 = x_2^+ - x_2$.

In the following, we now build the training data sets.

In [5]:
# Training outputs: Differences x_{k+1}-x_k
train_out = np.diff(x_train, axis=1)

# Training inputs: [x_k, u_k]
train_in = np.concatenate((x_train[:, :-1], u_train.T), axis=0)

The size of the training data sets is quite large at the moment. Usually, we do not need all the available training data to build a GP model with sufficiently well prediction capability. Especially training data that are very close to others can often be removed.

In the following, we use a simple distance-based approach for reducing the size of the training data set. We define a Euclidean distance that two training inputs at least need to have to each other to be considered as sufficiently different to provide new beneficial information. If the distance between two training data points is below this threshold distance, they are considered to provide approximately the same information and one of them is removed.

In [6]:
# Distance threshold: If inputs are too close, they are considered to be too similar and are removed for performance
# reasons
distThreshold = 8  # Minimum Euclidean distance to surrounding data points

# Iterate over all training data points and remove those that are not needed w.r.t. to the defined distance threshold
k = 0
while k < train_in.shape[1] - 1:
    # Select i-th data point as reference to compute pairwise distances to
    in_ref = train_in[:, k]
    out_ref = train_out[:, k]

    # Collect remaining data points that have not been used as reference yet
    in_remain = train_in[:, k + 1:]
    out_remain = train_out[:, k + 1:]

    # Compute pairwise distances to reference point
    dist = in_remain - mat.repmat(in_ref, in_remain.shape[1], 1).T  # Difference vectors
    dist = np.square(dist)  # Square elementwise
    dist = np.sum(dist, axis=0)  # Sum up row-wise
    dist = np.sqrt(dist)  # Taking square root

    # Get Boolean index for data points that are far enough to the reference point
    idx_keep = dist >= distThreshold

    # Keep only those remaining data points that are far enough from the current reference
    # (and therewith also from all previous ones)
    in_remain = in_remain[:, idx_keep]
    out_remain = out_remain[:, idx_keep]

    # Update training data set
    train_in = np.concatenate((train_in[:, :k], in_remain), axis=1)
    train_out = np.concatenate((train_out[:, :k], out_remain), axis=1)

    # Update running variable
    k += 1

Having processed the training data sets, we now define the GPs as described above and perform hyperparameter learning based on the training data.

In [7]:
# Define the GPs
kernel = kernels.SquaredExponential(variance=0.0001, bounds={'length_scales': (0.0001, 1e1), 'variance': (0.0001, 1e1)})

gp_1 = GPR(['x_0', 'x_1', 'u'], ['delta_x_0'], kernel=kernel)

kernel = kernels.SquaredExponential(variance=0.0001, bounds={'length_scales': (0.0001, 1e1), 'variance': (0.0001, 1e1)})

gp_2 = GPR(['x_0', 'x_1', 'u'], ['delta_x_1'], kernel=kernel)

gp_1.set_training_data(train_in, np.atleast_2d(train_out[0, :]))
gp_2.set_training_data(train_in, np.atleast_2d(train_out[1, :]))

gp_1.setup()
gp_2.setup()

gp_1.fit_model()
gp_2.fit_model()

We can plot the GPs after training simply by using the plot method:

In [8]:
gp_1.plot(output_notebook=True)

Loading BokehJS ...

In [9]:
gp_2.plot(output_notebook=True)

Loading BokehJS ...

### Prediction Model

Once the GPs are defined, we can set-up the prediction model for the MPC. 

We use a discretized model of the system, whose dynamics are given by

\begin{align}
\hat{x}_1^+ = \hat{x}_1 + \Delta x_1 \\
\hat{x}_2^+ = \hat{x}_2 + \Delta x_2 
\end{align}

where $\Delta x_1$ and $\Delta x_2$ denote the respective GP's one-step ahead mean prediction and $\hat{x}_i~,~~i \in \{ 1, 2 \}$ indicates the approximative character of the prediction model.

We set up the discrete model as written above, first, and afterwards, we replace the variables $\Delta x_1$ and $\Delta x_2$ by the respective GP's prediction.

In [10]:
# Make GP model for controller
gp_model = Model(plot_backend='bokeh', name='GP_Model', discrete=True)

# Set states and inputs
gp_model.set_dynamical_states('x', 2)
gp_model.set_inputs('u', 1)

# Set difference equations
gp_model.set_dynamical_equations(['x_0', 'x_1'])

# Substitute GPs
gp_model += [gp_1, gp_2]

# Set up the model
gp_model.setup(dt=Ts)

../../../hilo_mpc/core/model.py:2762: UserWarning: The model was initialized as a discrete model, but no sampling time was implemented in the model. Assuming that the sampling time 'dt' is already integrated numerically.
  warnings.warn("The model was initialized as a discrete model, but no sampling time was implemented in "


### Model Predictive Controller

Given the prediction model, we now create the model predictive controller to steer the system towards the  the reference  $(x_1^{\text{ref}}, x_2^{\text{ref}}) = (1,0)$. We use a quadratic stage and terminal cost, whereupon the terminal penalty is chosen according to the LQR. For simplicity, we assume that there are neither input nor state constraints. The horizon is set to $N = 15$. 

The controller is then described by

\begin{align}
&\mathbf{u} = \arg \min_{\mathbf{u}} \left \{ \sum_{k=0}^{N-1} \lVert \hat{x}_k \rVert _Q^2  + \lVert x_N \rVert_P^2 \right \} \\
&\text{s. t. } \hat{x}_{k+1} = \hat{x}_k + \Delta x_k ~,~~ \hat{x}_0 = x_0
\end{align}

where $Q = \begin{bmatrix} 100 & 0 \\ 0 & 100 \end{bmatrix}$, $R = 10$ and $P$ is the terminal penalty according to the LQR.

Solving the optimization problem will result in a N-step ahead control action sequence, whereof the first element is implemented throughout the next sampling period. Once the next measurement is available, the optimization problem will be resolved with the newest measurement as initial condition $x_0$.

**NOTE**: when using a GP into the MPC, it is recomandable to select the option `'ipopt.hessian_approximation': 'limited-memory'`. This will activate the numerical computation of the hessian istead of computing it symbolically, hence decreasing the memory requirment and computational time, at the price of possible numerical errors during the hessian computation.

In [11]:
# Make controller
gp_nmpc = NMPC(gp_model)

# Set horizon
gp_nmpc.horizon = 15

# Set cost function
gp_nmpc.quad_stage_cost.add_states(names=['x_0', 'x_1'], weights=[100, 100], ref=[1, 0])

gp_nmpc.quad_terminal_cost.add_states(names=['x_0', 'x_1'], weights=np.array([[8358.1, 1161.7], [1161.7, 2022.9]]),
                                      ref=[1, 0])

gp_nmpc.set_box_constraints(u_ub=u_ub, u_lb=u_lb)
# Set-up controller
gp_nmpc.setup(solver_options={'ipopt.hessian_approximation': 'limited-memory'},
              options={'print_level': 0})

../../../hilo_mpc/core/controller.py:490: UserWarning: The integration method is set to collocation but I notice that the model is in discrete time. I am overwriting and using discrete mode.
  default_opts.get('integration_method')))


## Simulation 
### Machine learning model

Finally, we can simulate the closed-loop system.

In [12]:
# Reset solution data frame for the true system
system.reset_solution()

# Vector of simulation time points
Tf = 20                     # Final time
t = np.arange(0, Tf, Ts)

scl = SimpleControlLoop(system, gp_nmpc)
scl.run(int(Tf/Ts))
scl.plot(output_notebook=True, title=['x0','x1','u'])

AttributeError: 'TimeSeries' object has no attribute 'backend'

## Acknowledgments
Thanks to Maik Pfefferkorn for providing this example.
